In [1]:
import numpy as np
import pandas as pd
np.random.seed(7)

In [2]:
def toNumber(inputList):
    outList = []    
    for word in inputList:
        newWord = []
        for char in word:
            newWord.append(ord(char))
        outList.append(newWord)          
    return outList

def fitToSize(word, n):
    wordFit = word[:n]
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)    
    return wordFit

def fixSymbolEncode(x):
    
    if x == 8217:
        return 39
    elif x == 8220:
        return 34
    elif x == 8221:
        return 34
    elif x == 8230:
        return 46
    elif x == 8722:
        return 45
    elif x == 8211:
        return 45
    elif x == 8216:
        return 39
    elif x == 8226:
        return 46
    elif x == 8364:
        return 36
    return x

In [3]:
data = pd.read_csv('dates.csv')
Y = np.array(data.word)
X = np.array(data.label)

In [4]:
for i in range(len(X)):
    if type(X[i]) != str:
        X[i] = ''
for x in X:
    if type(x) != str:        
        print(x)

In [5]:
data = toNumber(X)

In [6]:
for i in range(len(data)):
    data[i] = fitToSize(data[i],8)

In [7]:
for i in range(len(data)):
    for j in range(len(data[i])):        
            data[i][j] = fixSymbolEncode(data[i][j])

In [8]:
maxl = 0;
maxa = []
for i in range(len(data)):
    for j in range(len(data[i])):        
           if data[i][j] > 126:
                maxl+=1                
print(maxl)

3042


In [7]:
labels = []
for y in Y:
    if y == 'NONE':
        labels.append(0)
    elif y == 'D_ST':
        labels.append(2)
    elif y == 'D_PT':
        labels.append(1)

In [14]:
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
from keras.models import Model


In [31]:
inputLayer = Input(dtype='int32', shape=(None,8,), name = 'inputLayer')
embeddingLayer = TimeDistributed(Embedding(input_dim = 9000, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer'), name='TDEmbedding')(inputLayer)
lstmLayer = TimeDistributed(LSTM(50, return_sequences=False, consume_less='gpu', 
                                name = 'lstmLayer'), name='TDlstm')(embeddingLayer)
outputLayer = TimeDistributed(Dense(output_dim=1,init='zero',activation='softmax',name='outputLayer'),
                              name='TDout')(lstmLayer)
model = Model(input=inputLayer, output=outputLayer)
model.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy','fmeasure'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
inputLayer (InputLayer)          (None, None, 8)       0                                            
____________________________________________________________________________________________________
TDEmbedding (TimeDistributed)    (None, None, 8, 10)   90000       inputLayer[0][0]                 
____________________________________________________________________________________________________
TDlstm (TimeDistributed)         (None, None, 50)      12200       TDEmbedding[0][0]                
____________________________________________________________________________________________________
TDout (TimeDistributed)          (None, None, 1)       51          TDlstm[0][0]                     
Total params: 102,251
Trainable params: 102,251
Non-trainable params: 0
___________________

In [15]:
data = np.array(data)
data.shape

(459952, 8)

In [69]:
data.shape

(459952, 8)

In [21]:
trainX = data[:459950]
trainX = trainX.reshape(45995,10,8)
trainX.shape

(45995, 10, 8)

In [39]:
trainX[:2]

array([[[ 82, 117, 115, 115, 105,  97,  39, 115],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [105, 110,   0,   0,   0,   0,   0,   0],
        [ 99, 114, 105, 115, 105, 115,   0,   0],
        [100, 117, 101,   0,   0,   0,   0,   0],
        [116, 111,   0,   0,   0,   0,   0,   0],
        [109,  97, 110,  97, 103, 101, 109, 101],
        [105, 110, 101, 102, 102, 105,  99, 105],
        [ 77,  79,  83,  67,  79,  87,   0,   0],
        [ 77,  97, 114,   0,   0,   0,   0,   0]],

       [[ 49,  57,   0,   0,   0,   0,   0,   0],
        [ 40,  80,  82,  73,  77,  69,  41,   0],
        [ 45,  45,   0,   0,   0,   0,   0,   0],
        [ 82, 117, 115, 115, 105,  97, 110,   0],
        [112, 111, 119, 101, 114,   0,   0,   0],
        [103, 114, 105, 100,   0,   0,   0,   0],
        [ 99, 111, 109, 112,  97, 110, 121,   0],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [ 97,   0,   0,   0,   0,   0,   0,   0],
        [117, 110, 105, 116,   0,   0,   0,   0]

In [22]:
trainY = np.array(labels[:459950])
trainY = trainY.reshape(45995,10,1)
trainY.shape

(45995, 10, 1)

In [38]:
trainY[:2]

array([[[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [2]],

       [[1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]])

In [32]:
model.fit(trainX[:10000], trainY[:10000])

Epoch 1/10
10000/10000 [==============================] - 9s - loss: 0.9963 - acc: 0.0037 - fmeasure: 0.0382      

In [33]:
answer = model.predict(trainX[10000:20000])

In [35]:
answer[10:20]

array([[[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]

In [18]:
testdf = pd.DataFrame(answer, labels)
testdf.to_csv('testdf.csv')